In [10]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
^C


In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [8]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Exception: Unable to find py4j in /content/spark-3.5.1-bin-hadoop3/python, your SPARK_HOME may not be configured correctly

In [ ]:
#ruta_base = "file:/home/vagrant/"
data = spark.sparkContext.textFile("enfermedades.csv").map(lambda x: x.split(";")).cache()
data.take(5)

[['DOCUMENTO_ANONIMIZADO',
  'PERIODO',
  'MES',
  'RENAES',
  'IPRESS',
  'REGION',
  'DEPARTAMENTO',
  'PROVINCIA',
  'DISTRITO',
  'UBIGEO',
  'CODIGO_DIAGNOSTICO',
  'DIAGNOSTICOS',
  'GRUPO_DIAGNOSTICOS',
  'GRUPO_COBERTURA',
  'SEXO',
  'EDAD',
  'TIPO_SEGURO',
  'SERVICIO',
  'FECHA_ATENCION',
  'MONTO_BRUTO',
  'FECHA_CORTE'],
 ['27549',
  '2022',
  '7',
  '16918',
  'INSTITUTO NACIONAL DE SALUD DEL NI�O - SAN BORJA',
  'LIMA',
  'Lima',
  'Lima',
  'Lima',
  '150101.0',
  'Z948',
  'ESTADO DE TRASPLANTE DE M�DULA �SEA',
  'TRASPLANTE DE PROGENITORES HEMATOPOY�TICOS',
  'PAC',
  'MASCULINO',
  '5',
  'SIS GRATUITO',
  'AMBULATORIO',
  '20220627',
  '0.22',
  '20230525'],
 ['31973',
  '2022',
  '6',
  '1703',
  'ELEAZAR GUZMAN BARRON',
  'ANCASH',
  'Ancash',
  'Santa',
  'Nuevo Chimbote',
  '21809.0',
  'E119',
  'Diabetes mellitus no insulinodependiente, sin menci�n de complicaci�n (MODY)',
  'ENFERMEDAD RARA O HUERFANA',
  'ERH',
  'MASCULINO',
  '68',
  'SIS GRATUITO',
  'AM

In [ ]:

# Obtenemos la primera fila del RDD
encabezado = data.first()

# Imprimimos el encabezado
print(encabezado)

['DOCUMENTO_ANONIMIZADO', 'PERIODO', 'MES', 'RENAES', 'IPRESS', 'REGION', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'UBIGEO', 'CODIGO_DIAGNOSTICO', 'DIAGNOSTICOS', 'GRUPO_DIAGNOSTICOS', 'GRUPO_COBERTURA', 'SEXO', 'EDAD', 'TIPO_SEGURO', 'SERVICIO', 'FECHA_ATENCION', 'MONTO_BRUTO', 'FECHA_CORTE']


In [ ]:

from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType, IntegerType

estructura = StructType([
    StructField('documento_anonimizado', StringType(), True),
    StructField('periodo', IntegerType(), True),
    StructField('mes', IntegerType(), True),
    StructField('renaes', StringType(), True),
    StructField('ipress', StringType(), True),
    StructField('region', StringType(), True),
    StructField('departamento', StringType(), True),
    StructField('provincia', StringType(), True),
    StructField('distrito', StringType(), True),
    StructField('ubigeo', StringType(), True),
    StructField('codigo_diagnostico', StringType(), True),
    StructField('diagnosticos', StringType(), True),
    StructField('grupo_diagnosticos', StringType(), True),
    StructField('grupo_cobertura', StringType(), True),
    StructField('sexo', StringType(), True),
    StructField('edad', IntegerType(), True),
    StructField('tipo_seguro', StringType(), True),
    StructField('servicio', StringType(), True),
    StructField('fecha_atencion', DateType(), True),
    StructField('monto_bruto', FloatType(), True),
    StructField('fecha_corte', DateType(), True)
])



In [ ]:

# Para formatear las fechas
from datetime import datetime
def parse_date(date_string):
    if date_string:
        return datetime.strptime(date_string, '%Y%m%d').date()
    else:
        return None

filas = data.filter(lambda x: x != encabezado)

# Transformamos el RDD, aplicando el casteo de tipos a las columnas
filas = filas.map(lambda x: [
    x[0],
    int(x[1]),
    int(x[2]),
    x[3],
    x[4],
    x[5],
    x[6],
    x[7],
    x[8],
    x[9],
    x[10],
    x[11],
    x[12],
    x[13],
    x[14],
    int(x[15]),
    x[16],
    x[17],
    parse_date(x[18]),
    float(x[19]),
    parse_date(x[20])
])

filas.take(3)

[['27549',
  2022,
  7,
  '16918',
  'INSTITUTO NACIONAL DE SALUD DEL NI�O - SAN BORJA',
  'LIMA',
  'Lima',
  'Lima',
  'Lima',
  '150101.0',
  'Z948',
  'ESTADO DE TRASPLANTE DE M�DULA �SEA',
  'TRASPLANTE DE PROGENITORES HEMATOPOY�TICOS',
  'PAC',
  'MASCULINO',
  5,
  'SIS GRATUITO',
  'AMBULATORIO',
  datetime.date(2022, 6, 27),
  0.22,
  datetime.date(2023, 5, 25)],
 ['31973',
  2022,
  6,
  '1703',
  'ELEAZAR GUZMAN BARRON',
  'ANCASH',
  'Ancash',
  'Santa',
  'Nuevo Chimbote',
  '21809.0',
  'E119',
  'Diabetes mellitus no insulinodependiente, sin menci�n de complicaci�n (MODY)',
  'ENFERMEDAD RARA O HUERFANA',
  'ERH',
  'MASCULINO',
  68,
  'SIS GRATUITO',
  'AMBULATORIO',
  datetime.date(2022, 4, 25),
  13.46,
  datetime.date(2023, 5, 25)],
 ['8629',
  2022,
  3,
  '6216',
  'INSTITUTO NACIONAL DE SALUD DEL NI�O - BRE�A',
  'LIMA',
  'Lima',
  'Lima',
  'Bre�a',
  '150105.0',
  'Q750',
  'Craneosinostosis',
  'ENFERMEDAD RARA O HUERFANA',
  'ERH',
  'MASCULINO',
  0,
  'SIS

In [ ]:

df = spark.createDataFrame(filas, schema=estructura)



In [ ]:

df.show(3)






+---------------------+-------+---+------+--------------------+------+------------+---------+--------------+--------+------------------+--------------------+--------------------+---------------+---------+----+------------+-----------+--------------+-----------+-----------+
|documento_anonimizado|periodo|mes|renaes|              ipress|region|departamento|provincia|      distrito|  ubigeo|codigo_diagnostico|        diagnosticos|  grupo_diagnosticos|grupo_cobertura|     sexo|edad| tipo_seguro|   servicio|fecha_atencion|monto_bruto|fecha_corte|
+---------------------+-------+---+------+--------------------+------+------------+---------+--------------+--------+------------------+--------------------+--------------------+---------------+---------+----+------------+-----------+--------------+-----------+-----------+
|                27549|   2022|  7| 16918|INSTITUTO NACIONA...|  LIMA|        Lima|     Lima|          Lima|150101.0|              Z948|ESTADO DE TRASPLA...|TRASPLANTE DE PRO...|

In [ ]:

df.printSchema()

root
 |-- documento_anonimizado: string (nullable = true)
 |-- periodo: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- renaes: string (nullable = true)
 |-- ipress: string (nullable = true)
 |-- region: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- distrito: string (nullable = true)
 |-- ubigeo: string (nullable = true)
 |-- codigo_diagnostico: string (nullable = true)
 |-- diagnosticos: string (nullable = true)
 |-- grupo_diagnosticos: string (nullable = true)
 |-- grupo_cobertura: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- tipo_seguro: string (nullable = true)
 |-- servicio: string (nullable = true)
 |-- fecha_atencion: date (nullable = true)
 |-- monto_bruto: float (nullable = true)
 |-- fecha_corte: date (nullable = true)



In [ ]:
#DEPURACION
#1. VERIFICAR EL % DE FILAS CON VALORES NULOS O ESPACIOS VACIOS
from pyspark.sql import functions as F

# Calcular el porcentaje de datos faltantes (nulos o vacíos) para cada columna
porcentaje_faltantes = [(col, (df.filter((F.col(col).isNull()) | (F.col(col) == "") | (F.trim(F.col(col)) == "")).count() / df.count()) * 100) for col in df.columns]

# Convertir el resultado a un DataFrame sin redondeo
df_porcentaje = spark.createDataFrame(porcentaje_faltantes, ["Columna", "Porcentaje_Faltantes"])

# Mostrar el DataFrame con todos los decimales
df_porcentaje.show(21)

+--------------------+--------------------+
|             Columna|Porcentaje_Faltantes|
+--------------------+--------------------+
|documento_anonimi...|                 0.0|
|             periodo|                 0.0|
|                 mes|                 0.0|
|              renaes|                 0.0|
|              ipress|  0.2781289506953224|
|              region|                 0.0|
|        departamento| 0.28396382378683266|
|           provincia| 0.28396382378683266|
|            distrito| 0.28396382378683266|
|              ubigeo| 0.28396382378683266|
|  codigo_diagnostico|                 0.0|
|        diagnosticos|                 0.0|
|  grupo_diagnosticos|                 0.0|
|     grupo_cobertura|                 0.0|
|                sexo|                 0.0|
|                edad|                 0.0|
|         tipo_seguro|                 0.0|
|            servicio|                 0.0|
|      fecha_atencion|                 0.0|
|         monto_bruto|          

In [ ]:

from pyspark.sql import functions as F

print(f"Número de filas original: {df.count()}")

# Crear una condición combinada para identificar filas con valores nulos o vacíos
condicion = None
for col in df.columns:
    nueva_condicion = (F.col(col).isNull()) | (F.trim(F.col(col)) == "")  # Verificar nulos o espacios vacíos
    if condicion is None:
        condicion = nueva_condicion
    else:
        condicion = condicion | nueva_condicion

# Filtrar las filas que NO cumplen con la condición (es decir, eliminar las filas con nulos o vacíos)
df = df.filter(~condicion)

# Mostrar el número de filas después de eliminar las filas con nulos o vacíos
print(f"Número de filas después de la limpieza: {df.count()}")

Número de filas original: 51415
Número de filas después de la limpieza: 51126


In [ ]:

print(f"Número de filas original sin vacios: {df.count()}")
df = df[df['GRUPO_DIAGNOSTICOS'] == 'ENFERMEDAD RARA O HUERFANA']
print(f"Número de filas original con la ENFERMEDAD RARA: {df.count()}")
#print(df['GRUPO_DIAGNOSTICOS'].unique())

Número de filas original sin vacios: 51126
Número de filas original con la ENFERMEDAD RARA: 48184


In [ ]:

#1. DEPURACIÓN DE FORMATO Y CONSISTENCIA DE LAS FECHAS
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_date

# Depuración de formato y consistencia de las fechas
df = df.withColumn("FECHA_ATENCION_VALID", to_date(col("FECHA_ATENCION"), "yyyyMMdd").isNotNull())
df = df.withColumn("FECHA_CORTE_VALID", to_date(col("FECHA_CORTE"), "yyyyMMdd").isNotNull())

# Creación de una tabla resumen con la cantidad de fechas mal formateadas
df_summary = df.agg(
    F.sum(F.when(~col("FECHA_ATENCION_VALID"), 1).otherwise(0)).alias("Fechas_ATENCION_mal_formateadas"),
    F.sum(F.when(~col("FECHA_CORTE_VALID"), 1).otherwise(0)).alias("Fechas_CORTE_mal_formateadas"),
    F.count("*").alias("Total_filas")
)

df_summary.show()

+-------------------------------+----------------------------+-----------+
|Fechas_ATENCION_mal_formateadas|Fechas_CORTE_mal_formateadas|Total_filas|
+-------------------------------+----------------------------+-----------+
|                              0|                           0|      48184|
+-------------------------------+----------------------------+-----------+



In [ ]:

print(f"Número de filas original sin vacios y solo con ENFERMEDAD: {df.count()}")

#2. DEPURACION Verificar valores inconsistentes de las edades
df = df.filter((col("EDAD") >= 0) & (col("EDAD") <= 110))
print(f"Número de filas después de la limpieza de edad: {df.count()}")

#3. Verificar que MONTO_BRUTO sea un valor positivo
df=df.filter(col("MONTO_BRUTO") >= 0)
print(f"Número de filas después de la limpieza del monto: {df.count()}")

#4. DEPURACION Verificar valores inconsistentes de los meses
df=df.filter((col("MES") > 0) | (col("MES") <= 12))
print(f"Número de filas después de la limpieza del mes: {df.count()}")

#4. Verificar valores inconsistentes en la columna SEXO
df.select("SEXO").distinct().show()

#5. Verificar valores inconsistentes en la columna TIPO_SEGURO
df.select("TIPO_SEGURO").distinct().show()

#5. Verificar valores inconsistentes en la columna TIPO_SEGURO
df.select("REGION").distinct().show(24)

Número de filas original sin vacios y solo con ENFERMEDAD: 48184
Número de filas después de la limpieza de edad: 48184
Número de filas después de la limpieza del monto: 48184
Número de filas después de la limpieza del mes: 48184
+---------+
|     SEXO|
+---------+
| FEMENINO|
|MASCULINO|
+---------+

+-----------------+
|      TIPO_SEGURO|
+-----------------+
|  SIS EMPRENDEDOR|
|     SIS GRATUITO|
|SIS INDEPENDIENTE|
|   SIS PARA TODOS|
|SIS MICROEMPRESAS|
+-----------------+

+-------------+
|       REGION|
+-------------+
|     AYACUCHO|
|     APURIMAC|
|       CALLAO|
|  LA LIBERTAD|
|     AMAZONAS|
|      UCAYALI|
|       TUMBES|
|        JUNIN|
|     AREQUIPA|
|         PUNO|
|   LAMBAYEQUE|
|MADRE DE DIOS|
|      HUANUCO|
|        PIURA|
| HUANCAVELICA|
|       LORETO|
|    CAJAMARCA|
|     MOQUEGUA|
|   SAN MARTIN|
|        TACNA|
|        CUSCO|
|          ICA|
|       ANCASH|
|         LIMA|
+-------------+



In [ ]:

#6. Eliminar filas duplicadas, si es que lo hay
filas_antes = df.count()
df = df.dropDuplicates()
filas_despues = df.count()

print(f"Filas antes de eliminar duplicados: {filas_antes}")
print(f"Filas después de eliminar duplicados: {filas_despues}")
print(f"Cantidad de duplicados eliminados: {filas_antes - filas_despues}")



Filas antes de eliminar duplicados: 48184
Filas después de eliminar duplicados: 47645
Cantidad de duplicados eliminados: 539


In [ ]:

#Código de verificación

from pyspark.sql import functions as F

# Calcular el porcentaje de datos faltantes (nulos o vacíos) para cada columna
porcentaje_faltantes = [(col, (df.filter((F.col(col).isNull()) | (F.col(col) == "") | (F.trim(F.col(col)) == "")).count() / df.count()) * 100) for col in df.columns]

# Convertir el resultado a un DataFrame sin redondeo
df_porcentaje = spark.createDataFrame(porcentaje_faltantes, ["Columna", "Porcentaje_Faltantes"])

# Mostrar el DataFrame con todos los decimales
df_porcentaje.show(21)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:

# INDICADOR 1
from pyspark.sql import functions as F
from pyspark.sql.functions import sum as _sum, count, round, col

# INDICADOR 1: Agrupación por GRUPO_COBERTURA, SEXO y DIAGNOSTICOS con orden alfabético
resultado1 = df.groupBy("SEXO", "DIAGNOSTICOS") \
    .agg(
        count("*").alias("TOTAL_ATENCIONES"),
        round((_sum("MONTO_BRUTO") / count("*")), 2).alias("COSTO_PROMEDIO_ATENCION")
    ) \
    .orderBy("DIAGNOSTICOS")

print("COSTO PROMEDIO Y TOTAL DE ATENCIONES POR SEXO:")
resultado1.show(truncate=False)


In [ ]:

#INDICADOR 2
from pyspark.sql import functions as F

# Calcular el número de diagnósticos por sexo
diagnosticos_por_sexo = df.groupBy("SEXO").agg(
    F.count("DIAGNOSTICOS").alias("TOTAL_DIAGNOSTICOS")
)

# Calcular el total general de diagnósticos
total_diagnosticos = df.select(F.count("DIAGNOSTICOS").alias("TOTAL_GENERAL")).collect()[0]["TOTAL_GENERAL"]

# Calcular el porcentaje de diagnósticos por sexo
porcentaje_diagnosticos = diagnosticos_por_sexo.withColumn(
    "PORCENTAJE_DIAGNOSTICOS(%)",
    F.round((F.col("TOTAL_DIAGNOSTICOS") / total_diagnosticos) * 100, 2)
)

# Mostrar el resultado ordenado por SEXO
porcentaje_diagnosticos.orderBy("SEXO").show(truncate=False)


In [ ]:

#INDICADOR 3
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sum as _sum, count, round

# Calcular el costo promedio y el total de atenciones agrupando por tipo de seguro y diagnóstico
resultado3 = df.groupBy("TIPO_SEGURO", "DIAGNOSTICOS").agg(
    count("*").alias("TOTAL_ATENCIONES"),  # Total de atenciones
    round(_sum("MONTO_BRUTO") / count("*"), 2).alias("COSTO_PROMEDIO")  # Costo promedio usando _sum
)

# Mostrar el resultado ordenado por tipo de seguro
resultado3.orderBy("TIPO_SEGURO").show(truncate=False)



In [ ]:

#INDICADOR 4
# Contamos el número total de atenciones en todas las regiones
from pyspark.sql import functions as F
from pyspark.sql.functions import col

total_atenciones = df.count()

# Contamos el número de atenciones por cada región
atenciones_por_region = df.groupBy("REGION").count().withColumnRenamed("count", "TOTAL_ATENCIONES")

# Calculamos la tasa de atención por región y redondeamos en una sola operación
tasa_atencion_region = atenciones_por_region.withColumn(
    "Tasa de Atención (%)",
    F.round((col("TOTAL_ATENCIONES") / total_atenciones) * 100, 2)
)

# Mostramos el resultado
tasa_atencion_region.show(24)


In [ ]:

#ESTADISTICA DESCRIPTIVA
from pyspark.sql.functions import round

# Calcular las estadísticas numéricas
numerical_stats = df.select("EDAD", "MONTO_BRUTO") \
    .summary("count", "mean", "stddev", "min", "max")

# Redondear a 2 decimales
numerical_stats_rounded = numerical_stats.select(
    "summary",
    round("EDAD", 2).alias("EDAD"),
    round("MONTO_BRUTO", 2).alias("MONTO_BRUTO")
)

# Mostrar el resultado
numerical_stats_rounded.show()


In [ ]:

#CORRELACION ENTRE DATOS
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml.stat import Correlation

# Preparar datos para correlación
df = df.withColumn("EDAD", col("EDAD").cast("integer"))
df = df.withColumn("MONTO_BRUTO", col("MONTO_BRUTO").cast("float"))

vector_cols = ["EDAD", "MONTO_BRUTO"]
assembler = VectorAssembler(inputCols=vector_cols, outputCol="features")
df_vector = assembler.transform(df)


# Calcular matriz de correlación
matrix = Correlation.corr(df_vector, "features").head()[0]
correlation_matrix = matrix.toArray().tolist()

for row in correlation_matrix:
    print(row)

In [ ]:

#ANALISIS ESTADISTICO
# KPI 1: Análisis detallado
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, round, avg, stddev, sum as _sum, when

# Calcular métricas detalladas
resultado1_detallado = df.groupBy("SEXO", "DIAGNOSTICOS") \
    .agg(
        count("*").alias("TOTAL_ATENCIONES"),  # Total de atenciones
        round(avg("MONTO_BRUTO"), 2).alias("COSTO_PROMEDIO_ATENCION"),  # Promedio de MONTO_BRUTO
        # Si el total de atenciones es 1, la desviación estándar será 0
        round(when(count("*") == 1, 0).otherwise(stddev("MONTO_BRUTO")), 2).alias("DESVIACION_ESTANDAR_MONTO"),
        round(avg("EDAD"), 2).alias("PROMEDIO_EDAD")  # Promedio de la edad
    ) \
    .orderBy("DIAGNOSTICOS")

# Mostrar resultados
print("ANÁLISIS DETALLADO: COSTO PROMEDIO, DESVIACIÓN ESTÁNDAR Y PROMEDIO DE EDAD POR SEXO Y DIAGNÓSTICO:")
resultado1_detallado.show(truncate=False)


In [ ]:

#INDICADOR 2: ANALISIS DETALLADO
#INDICADOR 2: SEXO, TOTAL DE DIAGNOSTICOS, PORCENTAJE, EDAD(PROMEDIO), MONTO_BRUTO(PROMEDIO Y DES.EST)
from pyspark.sql import functions as F

# Calcular el número de diagnósticos, edad promedio, monto promedio y desviación estándar por sexo
diagnosticos_detallado = df.groupBy("SEXO").agg(
    F.count("DIAGNOSTICOS").alias("TOTAL_DIAGNOSTICOS"),               # Total de diagnósticos
    F.round(F.avg("EDAD"), 2).alias("EDAD_PROMEDIO"),                  # Promedio de edad
    F.round(F.avg("MONTO_BRUTO"), 2).alias("PROMEDIO_MONTO_BRUTO"),    # Promedio del monto bruto
    F.round(F.stddev("MONTO_BRUTO"), 2).alias("DESVIACION_MONTO_BRUTO")# Desviación estándar del monto bruto
)

# Calcular el total general de diagnósticos
total_diagnosticos = df.select(F.count("DIAGNOSTICOS").alias("TOTAL_GENERAL")).collect()[0]["TOTAL_GENERAL"]

# Calcular el porcentaje de diagnósticos por sexo
diagnosticos_con_porcentaje = diagnosticos_detallado.withColumn(
    "PORCENTAJE_DIAGNOSTICOS(%)",
    F.round((F.col("TOTAL_DIAGNOSTICOS") / total_diagnosticos) * 100, 2)
)

# Mostrar el resultado ordenado por SEXO
diagnosticos_con_porcentaje.orderBy("SEXO").show(truncate=False)



In [ ]:

#INDICADOR 3: ANALISIS DETALLADO
#INDICADOR 3: TIPO DE SEGURO, DIAGNOSTICO, MONTO(PROMEDIO Y DESV.), TOTAL DE ATENCIONES, EDAD(PROMEDIO)
from pyspark.sql import functions as F
from pyspark.sql.functions import col, sum as _sum, count, round, avg, stddev, when

# Calcular el costo promedio, la desviación estándar y el promedio de edad agrupando por tipo de seguro y diagnóstico
resultado_detallado = df.groupBy("TIPO_SEGURO", "DIAGNOSTICOS").agg(
    count("*").alias("TOTAL_ATENCIONES"),  # Total de atenciones
    round(avg("MONTO_BRUTO"), 2).alias("PROMEDIO_MONTO_BRUTO"),  # Promedio del MONTO_BRUTO
    # Si el total de atenciones es 1, la desviación estándar será 0
    round(when(count("*") == 1, 0).otherwise(stddev("MONTO_BRUTO")), 2).alias("DESVIACION_MONTO_BRUTO"),
    round(avg("EDAD"), 2).alias("PROMEDIO_EDAD")  # Promedio de la edad
)

# Mostrar el resultado ordenado por tipo de seguro y diagnóstico
resultado_detallado.orderBy("TIPO_SEGURO", "DIAGNOSTICOS").show(truncate=False)



In [ ]:

#INDICADOR 4: ANALISIS DETALLADO
#INDICADOR 4: REGION, TOTAL DE ATENCIONES, PORCENTAJE, EDAD(PROMEDIO), MONTO_BRUTO(PROMEDIO Y DES. ESTA)
from pyspark.sql import functions as F
from pyspark.sql.functions import col, avg, stddev, round, count, when

# Contar el número total de atenciones en todas las regiones
total_atenciones = df.count()

# Agrupamos por región, calculamos las métricas y ajustamos la desviación estándar si hay solo una atención
analisis_region = df.groupBy("REGION").agg(
    count("*").alias("TOTAL_ATENCIONES"),
    round(avg("EDAD"), 2).alias("PROMEDIO_EDAD"),
    round(avg("MONTO_BRUTO"), 2).alias("PROMEDIO_MONTO_BRUTO"),
    round(when(count("*") == 1, 0).otherwise(stddev("MONTO_BRUTO")), 2).alias("DESVIACION_MONTO_BRUTO")
).withColumn(
    "PORCENTAJE_ATENCIONES(%)",
    round((col("TOTAL_ATENCIONES") / total_atenciones) * 100, 2)
).orderBy(col("PORCENTAJE_ATENCIONES(%)").desc())

# Mostrar el resultado
analisis_region.show(truncate=False)


In [ ]:

#GRAFICOS
#INDICADOR 1
import matplotlib.pyplot as plt
# Convertir el DataFrame de Spark en un DataFrame de pandas
data_kpi = resultado1.orderBy("COSTO_PROMEDIO_ATENCION", ascending=False).toPandas()

# Seleccionar los 20 diagnósticos con mayor costo promedio
top_data = data_kpi.head(20)

# Crear el gráfico de barras horizontales, diferenciando por SEXO
plt.figure(figsize=(12, 8))

# Colorear las barras según el SEXO (azul para masculino, rosa para femenino)
for sexo in top_data['SEXO'].unique():
    subset = top_data[top_data['SEXO'] == sexo]
    plt.barh(subset["DIAGNOSTICOS"], subset["COSTO_PROMEDIO_ATENCION"], label=sexo, alpha=0.7)

# Añadir etiquetas y título
plt.xlabel("Costo Promedio de Atención")
plt.ylabel("Diagnósticos")
plt.title("Top 20 Diagnósticos con Mayor Costo Promedio de Atención por Sexo")

# Invertir el eje Y para que el diagnóstico con mayor costo esté arriba
plt.gca().invert_yaxis()

# Añadir una leyenda para SEXO
plt.legend(title="Sexo")

plt.tight_layout()
plt.show()


In [ ]:

# INDICADOR 2:
import matplotlib.pyplot as plt

# Convertir el DataFrame de PySpark a Pandas
data_sexo = porcentaje_diagnosticos.orderBy("SEXO").toPandas()

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Ajusté el tamaño para mejor visualización
plt.bar(data_sexo["SEXO"], data_sexo["PORCENTAJE_DIAGNOSTICOS(%)"], color="salmon")

# Etiquetas y título
plt.xlabel("Sexo")
plt.ylabel("Porcentaje de Diagnósticos (%)")
plt.title("Porcentaje de Diagnósticos por Sexo")

# Mostrar el gráfico
plt.xticks(rotation=0)  # No es necesario rotar ya que solo son dos categorías
plt.tight_layout()
plt.show()


In [ ]:

# Convertir el DataFrame de Spark en un DataFrame de pandas
data_kpi3_1 = resultado3.orderBy("COSTO_PROMEDIO", ascending=False).toPandas()

# Seleccionar los 20 diagnósticos con mayor costo promedio
top_data3_1 = data_kpi3_1.head(20)

# Crear el gráfico de barras agrupadas por TIPO_SEGURO para cada DIAGNOSTICO
plt.figure(figsize=(14, 8))

# Crear barras agrupadas por TIPO_SEGURO para cada DIAGNOSTICO
for tipo_seguro in top_data3_1["TIPO_SEGURO"].unique():
    subset = top_data3_1[top_data3_1["TIPO_SEGURO"] == tipo_seguro]
    plt.barh(subset["DIAGNOSTICOS"], subset["COSTO_PROMEDIO"], label=tipo_seguro, alpha=0.7)

# Añadir etiquetas y título
plt.xlabel("Costo Promedio")
plt.ylabel("Diagnósticos")
plt.title("Top 20 Diagnósticos con Mayor Costo Promedio por Tipo de Seguro")

# Invertir el eje Y para que el diagnóstico con mayor costo esté arriba
plt.gca().invert_yaxis()

# Añadir una leyenda para los tipos de seguro
plt.legend(title="Tipo de Seguro")

plt.tight_layout()
plt.show()

In [ ]:

# INDICADOR 4:
# Mostrar tasa de atención por "REGION"
data_region = tasa_atencion_region.orderBy("Tasa de Atención (%)").toPandas()
plt.figure(figsize=(10, 6))
plt.bar(data_region["REGION"], data_region["Tasa de Atención (%)"], color="violet")
plt.xlabel("Región")
plt.ylabel("Tasa de Atención (%)")
plt.title("Tasa de Atención por Región")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

#METRICAS DE RENDIMIENTO
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, round, avg, stddev

# Calcula las métricas de rendimiento global
performance_metrics = df.agg(
    countDistinct("DIAGNOSTICOS").alias("Total Diagnósticos"),  # Total de diagnósticos únicos
    round(avg("MONTO_BRUTO"), 2).alias("Promedio Monto Bruto"),  # Promedio del monto bruto
    round(stddev("MONTO_BRUTO"), 2).alias("Desviación Estándar Monto"),  # Desviación estándar del monto bruto
    round(avg("EDAD"), 2).alias("Promedio Edad"),  # Promedio de la edad
    count("*").alias("Total Atenciones") #Total de atenciones
).collect()[0].asDict()

print("\n=== Métricas de Rendimiento Global ===")
for field, value in performance_metrics.items(): #Iterar sobre el diccionario
    print(f"{field}: {value}")

In [ ]:

#MODELO PREDICTIVO
# 1. Distribución de la variable objetivo
if 'MONTO_BRUTO' in df.columns:
    df.groupBy('MONTO_BRUTO').count().show()





In [ ]:


# Lista de columnas a eliminar
columns_to_drop = ['DOCUMENTO_ANONIMIZADO', 'RENAES', 'DISTRITO', 'UBIGEO', 'SERVICIO', 'CODIGO_DIAGNOSTICO']

# Eliminar las columnas del DataFrame
df = df.drop(*columns_to_drop)


In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.functions import count, when, isnan, col

from pyspark.ml.feature import StringIndexer, VectorAssembler

from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.stat import Correlation
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml import Pipeline

#2. Análisis estadístico de columnas numéricas
df.describe().show()

# 4. Verificar correlaciones entre columnas numéricas
numeric_columns= [col_name for col_name, dtype in df.dtypes if dtype in ['int', 'double', 'float']]
if numeric_columns:
    assembler = VectorAssembler(inputCols=numeric_columns, outputCol="features")
    df_assembled = assembler.transform(df)
    correlation_matrix = Correlation.corr(df_assembled, "features").head()[0]
    print(f"Matriz de correlaciones (ordenada por columnas numércias):")
    print (correlation_matrix)



+-------+--------------------+------------------+--------------------+--------+------------+---------+--------------------+--------------------+---------------+---------+------------------+---------------+-----------------+
|summary|             periodo|               mes|              ipress|  region|departamento|provincia|        diagnosticos|  grupo_diagnosticos|grupo_cobertura|     sexo|              edad|    tipo_seguro|      monto_bruto|
+-------+--------------------+------------------+--------------------+--------+------------+---------+--------------------+--------------------+---------------+---------+------------------+---------------+-----------------+
|  count|               47645|             47645|               47645|   47645|       47645|    47645|               47645|               47645|          47645|    47645|             47645|          47645|            47645|
|   mean|              2022.0| 4.233518732290902|                NULL|    NULL|        NULL|     NULL|  

In [ ]:
# Lista de columnas a eliminar
columns_to_drop = ['DOCUMENTO_ANONIMIZADO', 'PERIODO', 'RENAES', 'DISTRITO', 'UBIGEO', 'SERVICIO', 'CODIGO_DIAGNOSTICO', 'ipress']

# Eliminar las columnas del DataFrame
df = df.drop(*columns_to_drop)

# 13. Preparación de los datos para el modelo de regresión
# Definir las columnas categóricas y numéricas
categorical_cols = ['sexo', 'tipo_seguro','region','departamento','provincia','grupo_diagnosticos'] #se saca ipress de las columnas categoricas
numerical_cols = ['edad']

# Crear StringIndexers para las columnas categóricas
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index", handleInvalid="keep") for col in categorical_cols]

# Crear VectorAssembler para combinar las características
assembler_inputs = [col + "_index" for col in categorical_cols] + numerical_cols
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Crear un Pipeline para transformar los datos
pipeline = Pipeline(stages=indexers + [assembler])

# Ajustar el Pipeline a los datos y transformar los datos
pipeline_model = pipeline.fit(df)
df_transformed = pipeline_model.transform(df)

# Seleccionar las columnas relevantes para el modelo
model_df = df_transformed.select("features", col("MONTO_BRUTO").alias("label"))  # "label" es el nombre estándar para la variable objetivo

# Mostrar los datos transformados
model_df.show(5)

# 14. Entrenar y evaluar múltiples modelos de regresión
# Dividir los datos en conjuntos de entrenamiento y prueba
train_data, test_data = model_df.randomSplit([0.7, 0.3], seed=1234)

# Definir los modelos de regresión
lr = LinearRegression(featuresCol="features", labelCol="label")
dt = DecisionTreeRegressor(featuresCol="features", labelCol="label", maxBins=72)  # Aumentar maxBins
rf = RandomForestRegressor(featuresCol="features", labelCol="label", numTrees=100, maxBins=72)  # Aumentar maxBins
gbt = GBTRegressor(featuresCol="features", labelCol="label", maxIter=100) # GBT does not have maxBins

# Entrenar los modelos
lr_model = lr.fit(train_data)
dt_model = dt.fit(train_data)
rf_model = rf.fit(train_data)
gbt_model = gbt.fit(train_data)

# Hacer predicciones en los datos de prueba
lr_predictions = lr_model.transform(test_data)
dt_predictions = dt_model.transform(test_data)
rf_predictions = rf_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

# 15. Evaluar los modelos utilizando RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")  # RMSE es una métrica común para regresión

# Calcular RMSE para cada modelo
lr_rmse = evaluator.evaluate(lr_predictions)
dt_rmse = evaluator.evaluate(dt_predictions)
rf_rmse = evaluator.evaluate(rf_predictions)
gbt_rmse = evaluator.evaluate(gbt_predictions)

# Imprimir los resultados
print("Resultados de la evaluación:")
print(f"Linear Regression RMSE: {lr_rmse}")
print(f"Decision Tree RMSE: {dt_rmse}")
print(f"Random Forest RMSE: {rf_rmse}")
print(f"GBT RMSE: {gbt_rmse}")

# Calcular R-squared para cada modelo
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
lr_r2 = evaluator_r2.evaluate(lr_predictions)
dt_r2 = evaluator_r2.evaluate(dt_predictions)
rf_r2 = evaluator_r2.evaluate(rf_predictions)
gbt_r2 = evaluator_r2.evaluate(gbt_predictions)
print(f"Linear Regression R-squared: {lr_r2}")
print(f"Decision Tree R-squared: {dt_r2}")
print(f"Random Forest R-squared: {rf_r2}")
print(f"GBT R-squared: {gbt_r2}")

# 16. Seleccionar el mejor modelo y realizar predicciones
# Basado en los resultados, selecciona el modelo con el RMSE más bajo (o la métrica que consideres más importante).
# En este ejemplo, asumiremos que el Random Forest Regression es el mejor modelo.
# Puedes usar el modelo seleccionado para hacer predicciones sobre nuevos datos.
# Para hacer esto, debes transformar los nuevos datos usando el mismo Pipeline que usaste para entrenar el modelo.
# Ejemplo de cómo hacer predicciones con el mejor modelo (Random Forest):
best_model = rf_model  # Asumiendo que Random Forest es el mejor modelo

# Puedes guardar el mejor modelo para usarlo en el futuro

# best_model.save("ruta/al/modelo")

print("¡Modelo entrenado y evaluado!")

+--------------------+-----+
|            features|label|
+--------------------+-----+
|      (7,[6],[75.0])| 1.26|
|[0.0,0.0,8.0,8.0,...|  4.8|
|(7,[2,3,4],[16.0,...| 41.6|
|(7,[0,6],[1.0,61.0])| 16.2|
|(7,[0,6],[1.0,14.0])|10.69|
+--------------------+-----+
only showing top 5 rows



IllegalArgumentException: requirement failed: DecisionTree requires maxBins (= 32) to be at least as large as the number of values in each categorical feature, but categorical feature 4 has 39 values. Consider removing this and other categorical features with a large number of values, or add more training examples.

In [ ]:

#4. Evaluación de categorías en columnas categóricas
categorical_columns = [col_name for col_name, dtype in df.dtypes if dtype == 'string']
for col_name in categorical_columns:
    unique_count = df.select(col_name).distinct().count()
    print(f"Columna categórica {col_name}' tiene {unique_count} valores únicos.")


Columna categórica ipress' tiene 71 valores únicos.
Columna categórica region' tiene 24 valores únicos.
Columna categórica departamento' tiene 24 valores únicos.
Columna categórica provincia' tiene 38 valores únicos.
Columna categórica diagnosticos' tiene 430 valores únicos.
Columna categórica grupo_diagnosticos' tiene 1 valores únicos.
Columna categórica grupo_cobertura' tiene 1 valores únicos.
Columna categórica sexo' tiene 2 valores únicos.
Columna categórica tipo_seguro' tiene 5 valores únicos.


In [ ]:

#5. Recomendación preliminar del modelo basado en análisis de datos
if len(numeric_columns) > 3 and df.select('MONTO_BRUTO').distinct().count() == 2:
    print("Sugerencia: Modelos de clasificación binaria como Logistic Regression o Random Forest son adecuados.")
elif len(numeric_columns) > 3:
  print("Sugerencia: Modelos de regresión como Linear Regression podrían ser útiles.")
else:
  print("Sugerencia: Realiza más análisis para identificar relaciones complejas y considerar modelos no lineales.")

#ESTOS CODIGOS TENEMOS QUE HACER PARA VER QUE RECOMIENDA

Sugerencia: Modelos de regresión como Linear Regression podrían ser útiles.


In [ ]:
#6. ESTE CODIGO YA DEFINE QUE MODELO UTILIZAR
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']
df.show(5)
from pyspark.ml import Pipeline
# Define los indexers
indexers = [StringIndexer(inputCol=column, outputCol=column + "_index") for column in categorical_columns]
# Configura el pipeline
pipeline = Pipeline(stages=indexers)
# Ajusta y transforma los datos
df_transformed = pipeline.fit(df).transform(df)
df_transformed.show(5)
feature_columns = [col + "_index" for col in categorical_columns[:-1]] + ['balance', 'duration', 'campaign', 'pdays', 'previous']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_transformed = assembler.transform(df_transformed)
df_transformed = df_transformed.select("features", "deposit_index") # Selecciona las columnas necesarias
df_transformed.show(5)


In [ ]:

#7. PERO POR LO MENOS ESTO SI TENEMOS QUE HACER
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=1234)

#Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="deposit_index")
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)

#Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="deposit_index", numTrees=100)
rf_model = rf.fit(train_data)
rf_predictions = rf_model.transform(test_data)

#Evaluador de clasificación binaria
evaluator = BinaryClassificationEvaluator(labelCol="deposit_index", rawPredictionCol="rawPrediction")

#Métricas para Logistic Regression
lr_auc = evaluator.evaluate(lr_predictions)
print(f"Logistic Regression AUC: {lr_auc}")

#Métricas para Random Forest
rf_auc = evaluator.evaluate(rf_predictions)
print(f"Random Forest AUC: {rf_auc}")

#Evaluación de Accuracy
accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="deposit_index", metricName="accuracy")
lr_accuracy = accuracy_evaluator.evaluate(lr_predictions)
rf_accuracy = accuracy_evaluator.evaluate(rf_predictions)
print(f"Logistic Regression Accuracy: {lr_accuracy}")
print(f"Random Forest Accuracy: {rf_accuracy}")








In [ ]:
#8. MODELO QUE VA A PREDECIR
# Y ACA RECIEN TENEMOS QUE HACER EL MODELO PREDICTIVO



In [ ]:
%pyspark







In [ ]:
%pyspark




In [ ]:
%pyspark

